In [3]:
%run /home/analysis_user/New_trap_code/Tools/basic_packages.py

# Imports
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
import os
import BeadDataFile as BDF
import glob
import re
import scipy.signal as signal
import beadUtils as bu

ModuleNotFoundError: No module named 'beadUtils'

In [11]:
figDir = 'Figures/'
if figDir.strip('/') not in os.listdir():
    os.mkdir(figDir)
    print(f"Figure directory didn't exist. Made it just now at {figDir}")
plt.style.use(['fivethirtyeight', 'gvELOG'])
saveFig = False

In [3]:
fftDict = {'fs':5e3}
tFFT = 10 # seconds
# fftDict['nfft'] = int(fftDict['fs']*tFFT)
fftDict['win'] = ('tukey', 0.25)
fftDict['tFFT'] = 10

In [4]:
Year = "2022"
Month = "06" # "04"
Day = "07"   # "06"
baseDir = f"/home/data_transfer/data_from_windows/{Year}{Month}{Day}/ASMcal/"
excOff = baseDir+'InputsTerminated/'
# excOff = baseDir+'MokuConnected/'
Xexc = baseDir + 'Xexc/'
Yexc = baseDir + 'Yexc/'

In [5]:
fExc = 13.1415
excAmp = 50e-3/2 * 15 #mV, the factor of 15 gets us to the actual voltage applied to the ASM003 piezos.
# Calibration numbers from elog 337
xCal, yCal = 7.83e-6, 5.54e-6 # urad/V
excAmpX, excAmpY = xCal*excAmp*1.27000000e-02, yCal*excAmp*1.27000000e-02 # m, last factor is to convert to motion at trap focus

In [6]:
for jj, (ff1, ff2, ff3) in enumerate(zip(bu.orderedFilenames(excOff, prefix='ASMcal'), bu.orderedFilenames(Xexc, prefix='ASMcal'),
                                   bu.orderedFilenames(Yexc, prefix='ASMcal'))):
    fil1, fil2, fil3 = BDF.BeadDataFile(ff1), BDF.BeadDataFile(ff2), BDF.BeadDataFile(ff3)
    if jj==0:
        noiseLevel_X = bu.fft(fil1.xyz2[0,:], fExc, **fftDict) 
        noiseLevel_Y = bu.fft(fil1.xyz2[1,:], fExc, **fftDict)
        x2x = bu.fft(fil2.xyz2[0,:], fExc, **fftDict) 
        x2y = bu.fft(fil2.xyz2[1,:], fExc, **fftDict) 
        y2x = bu.fft(fil3.xyz2[0,:], fExc, **fftDict) 
        y2y = bu.fft(fil3.xyz2[1,:], fExc, **fftDict) 
    else:
        noiseLevel_X = np.hstack((noiseLevel_X, bu.fft(fil1.xyz2[0,:], fExc, **fftDict))) 
        noiseLevel_Y = np.hstack((noiseLevel_Y, bu.fft(fil1.xyz2[1,:], fExc, **fftDict)))
        x2x = np.hstack((x2x, bu.fft(fil2.xyz2[0,:], fExc, **fftDict))) 
        x2y = np.hstack((x2y, bu.fft(fil2.xyz2[1,:], fExc, **fftDict))) 
        y2x = np.hstack((y2x, bu.fft(fil3.xyz2[0,:], fExc, **fftDict))) 
        y2y = np.hstack((y2y, bu.fft(fil3.xyz2[1,:], fExc, **fftDict)))


In [7]:
QPDcal_X, QPDerr_X = np.mean(np.abs(x2x)/excAmpX), np.std(np.abs(x2x)/excAmpX)
QPDcal_Y, QPDerr_Y = np.mean(np.abs(y2y)/excAmpY), np.std(np.abs(y2y)/excAmpY)